In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from bot import sendMessage
    
from __future__ import print_function
import seaborn; seaborn.set()

In [3]:
r_train = pd.read_csv('data/r_train.csv', index_col=0)
r_test = pd.read_csv('data/r_test.csv', index_col=0)
r_target = pd.read_csv('data/target.csv', index_col=0)['0']

In [4]:
droplist = set(['ind_var6', 'saldo_medio_var17_hace3', 'delta_imp_trasp_var33_out_1y3', 
                'saldo_medio_var17_hace2', 'ind_var20_0', 'delta_imp_reemb_var33_1y3',
                'delta_imp_reemb_var13_1y3', 'num_var13_medio_0', 'ind_var20', 'ind_var18_0',
                'delta_imp_amort_var18_1y3', 'delta_imp_amort_var34_1y3', 'saldo_medio_var12_ult1',
                'delta_imp_aport_var13_1y3', 'delta_imp_venta_var44_1y3', 'saldo_medio_var17_ult3',
                'num_meses_var13_medio_ult3', 'delta_num_aport_var33_1y3', 'delta_num_aport_var17_1y3',
                'delta_num_compra_var44_1y3', 'imp_reemb_var33_ult1', 'delta_imp_trasp_var17_out_1y3',
                'imp_reemb_var17_hace3', 'ind_var34_0', 'delta_imp_trasp_var17_in_1y3'])

droplist = ['imp_op_var39_efect_ult3', 'num_op_var41_ult3', 'imp_op_var39_efect_ult1', 
            'delta_imp_aport_var13_1y3', 'delta_imp_reemb_var33_1y3', 'num_trasp_var33_out_ult1', 
            'ind_var18_0', 'delta_imp_reemb_var13_1y3', 'saldo_medio_var13_medio_ult3', 
            'ind_var20_0', 'imp_trasp_var17_in_hace3', 'num_op_var41_ult1', 'num_var1_0', 
            'num_var44', 'ind_var40_0', 'ind_var13_largo_0', 'num_var34_0', 'ind_var24', 
            'num_op_var41_efect_ult1', 'saldo_var24', 'imp_op_var41_ult1', 'ind_var20', 
            'ind_var39_0', 'num_var40', 'num_trasp_var17_in_ult1', 'num_var8_0', 
            'saldo_medio_var17_ult3', 'ind_var6_0', 'num_var8', 'num_var6', 'num_var13_medio_0', 
            'num_var4', 'ind_var14', 'num_var1', 'saldo_var1', 'saldo_medio_var33_ult3', 
            'num_reemb_var17_hace3', 'num_op_var41_efect_ult3', 'delta_num_venta_var44_1y3', 
            'delta_num_aport_var33_1y3', 'saldo_medio_var17_hace3', 'saldo_medio_var17_hace2', 
            'ind_var34_0', 'saldo_medio_var33_ult1', 'num_var45_ult3', 'saldo_medio_var12_ult1', 
            'num_var18_0', 'saldo_medio_var13_medio_hace2', 'num_reemb_var33_ult1', 'ind_var13_corto_0', 
            'ind_var13', 'num_op_var41_hace3', 'num_op_var41_hace2', 'ind_var7_emit_ult1', 
            'num_var12', 'saldo_medio_var13_largo_ult1', 'ind_var24_0', 'saldo_var17', 
            'num_var5', 'delta_num_compra_var44_1y3', 'saldo_var26', 'saldo_var34', 
            'saldo_medio_var13_corto_ult1', 'saldo_var13_corto', 'saldo_var18', 
            'delta_num_aport_var17_1y3', 'num_meses_var13_medio_ult3', 'num_trasp_var17_out_ult1', 
            'ind_var44_0', 'imp_trasp_var33_out_ult1', 'num_var40_0', 'ind_var1', 'num_var13_corto']
droplist = []

In [5]:
data = r_train.drop(droplist, axis=1)

In [6]:
%%time
dc = data.corr()

Wall time: 21.6 s


In [13]:
#Find correlations
def bad_feature_pairs(corr, columns, th):
    bad_fts = dict()
    bad_count = {ft:0 for ft in columns}
    for ft1 in columns:
        for ft2 in columns:
            if ft1 == ft2: 
                continue
            if corr[ft1][ft2] > th:
                pair = tuple(sorted([ft1, ft2]))
                if pair not in bad_fts:
                    bad_fts[pair] = corr[ft1][ft2]
                    bad_count[ft1] += 1
                    bad_count[ft2] += 1
    return bad_fts, bad_count

corr_th = .95
bad_fts, bad_count = bad_feature_pairs(dc, dc.columns, corr_th)
print(len(bad_fts))

148


In [18]:
for fts in sorted(bad_fts.keys(), key=lambda x: -bad_fts[x]):
    print(fts[0], fts[1], bad_fts[fts])

ind_var6 num_var6 1.00000000001
delta_imp_trasp_var33_out_1y3 num_trasp_var33_out_ult1 1.0
delta_imp_reemb_var33_1y3 num_reemb_var33_ult1 1.0
ind_var20_0 num_var20_0 1.0
delta_imp_reemb_var13_1y3 num_reemb_var13_ult1 1.0
ind_var20 num_var20 1.0
ind_var13_medio_0 num_var13_medio_0 1.0
num_meses_var13_medio_ult3 num_var13_medio_0 1.0
delta_imp_amort_var18_1y3 num_var18_0 1.0
ind_var18_0 num_var18_0 1.0
delta_imp_amort_var34_1y3 num_var34_0 1.0
delta_imp_aport_var13_1y3 delta_num_aport_var13_1y3 1.0
delta_imp_venta_var44_1y3 delta_num_venta_var44_1y3 1.0
delta_imp_aport_var33_1y3 delta_num_aport_var33_1y3 1.0
delta_imp_aport_var17_1y3 delta_num_aport_var17_1y3 1.0
ind_var13_medio_0 num_meses_var13_medio_ult3 1.0
delta_imp_compra_var44_1y3 delta_num_compra_var44_1y3 1.0
delta_imp_amort_var18_1y3 ind_var18_0 1.0
imp_reemb_var33_ult1 num_reemb_var33_ult1 1.0
delta_imp_trasp_var17_out_1y3 num_trasp_var17_out_ult1 1.0
imp_reemb_var17_hace3 num_reemb_var17_hace3 1.0
ind_var34_0 num_var34_0 1.0


In [155]:
for ft in sorted(bad_count, key=lambda x: -bad_count[x]):
    if bad_count[ft] > 0:
        print(ft, bad_count[ft])

saldo_var17 6
saldo_medio_var17_hace2 6
saldo_medio_var17_ult1 6
saldo_medio_var17_ult3 6
saldo_medio_var17_hace3 6
imp_aport_var17_hace3 6
saldo_var31 6
saldo_var24 4
saldo_medio_var12_ult1 4
saldo_var12 4
num_var13_corto 4
num_var13_corto_0 4
saldo_medio_var13_medio_hace2 4
ind_var13_corto_0 4
ind_var13_corto 4
saldo_medio_var12_ult3 4
saldo_var42 4
num_meses_var13_corto_ult3 4
num_var13_medio_0 3
num_var40 3
num_var1 3
num_var40_0 3
ind_var1 3
ind_var40_0 3
num_meses_var13_medio_ult3 3
ind_var13_medio_0 3
ind_var13_0 3
num_var1_0 3
ind_var40 3
ind_var1_0 3
num_var13 3
delta_imp_reemb_var33_1y3 2
ind_var13 2
ind_var12 2
saldo_var18 2
imp_trasp_var33_out_ult1 2
saldo_medio_var33_ult1 2
num_var6 2
num_reemb_var33_ult1 2
saldo_medio_var33_ult3 2
saldo_var13_corto 2
ind_var6 2
saldo_medio_var13_medio_ult3 2
num_trasp_var33_out_ult1 2
saldo_var1 2
saldo_var6 2
ind_var34_0 2
num_var13_0 2
saldo_medio_var13_corto_ult1 2
saldo_medio_var13_corto_ult3 2
ind_var18_0 2
imp_amort_var18_ult1 2
num

In [19]:
correlated = set(map(lambda x: x[0], bad_fts) + map(lambda x: x[1], bad_fts))
len(correlated)

173

In [23]:
to_drop = set()
drop_type = 'identical'
while True:
    bf, bc = bad_feature_pairs(dc, set(dc.columns).difference(to_drop), corr_th)
    #print(bc)
    if drop_type == 'identical':
        ident = filter(lambda x: bf[x] >= 1., bf.keys())
        if len(ident) == 0:
            break
        to_drop.add(ident[0][0])
        print(ident[0][0], bf[ident[0]])
    else:
        bcs = sorted(bc, key=lambda x: -bc[x])
        print(bcs[0], bc[bcs[0]])
        if bc[bcs[0]] == 0:
            break
        to_drop.add(bcs[0])
print(len(to_drop), to_drop)

delta_imp_venta_var44_1y3 1.0
delta_imp_reemb_var33_1y3 1.0
ind_var13_medio_0 1.0
delta_imp_aport_var13_1y3 1.0
ind_var20 1.0
delta_imp_aport_var33_1y3 1.0
delta_imp_amort_var34_1y3 1.0
ind_var6 1.00000000001
delta_imp_aport_var17_1y3 1.0
ind_var18_0 1.0
delta_imp_trasp_var33_out_1y3 1.0
delta_imp_amort_var18_1y3 1.0
ind_var20_0 1.0
num_meses_var13_medio_ult3 1.0
delta_imp_reemb_var13_1y3 1.0
15 set(['delta_imp_aport_var13_1y3', 'delta_imp_reemb_var33_1y3', 'delta_imp_venta_var44_1y3', 'ind_var20', 'ind_var18_0', 'delta_imp_amort_var34_1y3', 'delta_imp_reemb_var13_1y3', 'delta_imp_aport_var17_1y3', 'ind_var20_0', 'num_meses_var13_medio_ult3', 'delta_imp_trasp_var33_out_1y3', 'delta_imp_amort_var18_1y3', 'ind_var13_medio_0', 'ind_var6', 'delta_imp_aport_var33_1y3'])


In [24]:
r_train.drop(to_drop, axis=1).to_csv('./data/c_train.csv')
r_test.drop(to_drop, axis=1).to_csv('./data/c_test.csv')